In [ ]:
# ------------------------------
# Setup paths and imports
# ------------------------------
import sys
from pathlib import Path

# Point to the src directory
project_root = Path().resolve().parents[1]  # adjust if notebook location changes
src_path = project_root / "src"
sys.path.append(str(src_path))

# ------------------------------
# Standard imports
# ------------------------------
from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter, ParameterVector
from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient, OperatorStateFn
from qiskit.utils import QuantumInstance

import matplotlib.pyplot as plt
import numpy as np

# ------------------------------
# Your local modules
# ------------------------------
from qiskit_utils import BinaryObjectiveFunction
from data_utils import circle, plot_data, generate_ds

# ------------------------------
# Random seed
# ------------------------------
np.random.seed(42)

# ------------------------------
# Example: Generate a toy dataset
# ------------------------------
X, y = generate_ds(n_samples=50, noise=0.1)
plot_data(X, y)




In [4]:
# Generate training and test data
num_training = 200
num_test = 2000

X_train, y_train, new_y_train = generate_ds(num_training)
X_test, y_test, new_y_test= generate_ds(num_test)

In [7]:
# set method to calculcate expected values
expval = AerPauliExpectation()

# define gradient method
gradient = Gradient()

# define quantum instances (statevector and sample based)
qi_sv = QuantumInstance(Aer.get_backend('aer_simulator_statevector'))

In [8]:
from qiskit_machine_learning.neural_networks import OpflowQNN

# construct parametrized circuit
inputs = ParameterVector('input', length=4) # 1 extra for label
weights = ParameterVector('weight', length=9)

# 1qubit classifier
qc1 = QuantumCircuit(1)
qc1.u(inputs[0],inputs[1],inputs[2], 0)
qc1.u(weights[0],weights[1],weights[2], 0)
qc1.u(inputs[0],inputs[1],inputs[2], 0)
qc1.u(weights[3],weights[4],weights[5], 0)
qc1.u(inputs[0],inputs[1],inputs[2], 0)
qc1.u(weights[6],weights[7],weights[8], 0)
qc_sfn1 = StateFn(qc1)

H1 = StateFn(PauliSumOp.from_list([('Z', 1.0)]))
H2 = StateFn(PauliSumOp.from_list([('Z', -1.0)]))

op1 = ~H1 @ (qc_sfn1)
op2 = ~H2 @ (qc_sfn1)
print(op1)

ComposedOp([
  OperatorMeasurement(1.0 * Z),
  CircuitStateFn(
     ┌───────────────────────────────┐┌──────────────────────────────────┐»
  q: ┤ U(input[0],input[1],input[2]) ├┤ U(weight[0],weight[1],weight[2]) ├»
     └───────────────────────────────┘└──────────────────────────────────┘»
  «   ┌───────────────────────────────┐┌──────────────────────────────────┐»
  «q: ┤ U(input[0],input[1],input[2]) ├┤ U(weight[3],weight[4],weight[5]) ├»
  «   └───────────────────────────────┘└──────────────────────────────────┘»
  «   ┌───────────────────────────────┐┌──────────────────────────────────┐
  «q: ┤ U(input[0],input[1],input[2]) ├┤ U(weight[6],weight[7],weight[8]) ├
  «   └───────────────────────────────┘└──────────────────────────────────┘
  )
])


In [9]:
# construct OpflowQNN with the operator, the input parameters, the weight parameters,
# the expected value, gradient, and quantum instance.
qnn1 = OpflowQNN(op1, inputs, weights, expval, gradient, qi_sv)
qnn2 = OpflowQNN(op2, inputs, weights, expval, gradient, qi_sv)

In [16]:
from qiskit.algorithms.optimizers import ADAM, L_BFGS_B
from qiskit_machine_learning.utils.loss_functions import L2Loss
from qiskit.utils import algorithm_globals
import numpy as np

# Track losses
losses = []

# Callback function
def callback_fn(avg_loss, weights):
    print("weights: ", weights)
    print("loss: ", avg_loss)
    losses.append(avg_loss)

# Initialize the objective function
function = BinaryObjectiveFunction(X_train, new_y_train, qnn1, qnn2, L2Loss(), callback_fn)

# Choose optimizer
optimizer = L_BFGS_B(maxiter=50)
# optimizer = ADAM(maxiter=30, lr=0.8)

# Initial random weights
initial_point = algorithm_globals.random.random(qnn1.num_weights)

# Run optimization
fit_result = optimizer.minimize(
    fun=function.objective,   # objective function
    x0=initial_point,         # starting point
    jac=function.gradient      # gradient function
)

# Optimized weights
optimal_weights = fit_result.x
print("Optimized weights:", optimal_weights)
print("Final loss:", fit_result.fun)


TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
plt.plot(losses)

In [3]:
ws = [-0.91709316,  1.57980935, -1.61740473,  1.1182353,  -1.56539397,  1.52962569,
 -1.52987669,  0.00201624,  1.47818211]

from test_utils import get_metrics
y_predict, y_tensor, accuracy, [tp, tn, fp, fn] = get_metrics(X_train, qnn1, ws)

ModuleNotFoundError: No module named 'test_utils'

In [13]:
# print('Accuracy:', acc/len(y_tensor))
plt.figure(figsize=(8,8))
for x, y_target, y_ in zip(X_train, y_tensor, y_predict):
    if y_ == 1:
        plt.plot(x[0], x[1], 'bo')
    else:
        plt.plot(x[0], x[1], 'go')
    if y_target != y_:
        plt.scatter(x[0], x[1], s=200, facecolors='none', edgecolors='r', linewidths=1)
plt.show()

NameError: name 'y_tensor' is not defined

<Figure size 800x800 with 0 Axes>

In [ ]:
y_predict, y_tensor, accuracy, [tp, tn, fp, fn] = get_metrics(X_test, qnn1, ws)

In [ ]:
# print('Accuracy:', acc/len(y_tensor))
plt.figure(figsize=(8,8))
for x, y_target, y_ in zip(X_test, y_tensor, y_predict):
    if y_ == 1:
        plt.plot(x[0], x[1], 'bo')
    else:
        plt.plot(x[0], x[1], 'go')
    if y_target != y_:
        plt.scatter(x[0], x[1], s=200, facecolors='none', edgecolors='r', linewidths=1)
plt.show()

: 